[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/SlimeVRX/SlimeVRX/blob/main/hipnuc/05.ipynb)

## Study [Attitude determination for IMU & AHRS](https://zhuanlan.zhihu.com/p/351596374) 
#**Lesson 05: Examples of 3D rotation.**

Giải thích thêm về hệ n (hệ tọa độ dẫn đường)

Kiến thức được đề cập trước đây chỉ để nói rằng hệ n thuộc hệ tọa độ cố định. Tìm hiểu thêm một chút về n-series tại đây.

Dòng n thường được sử dụng (hệ thống định vị):
- Hệ thống NED (XYZ tương ứng North-East-Down, thường được sử dụng ở nước ngoài) tương ứng với thứ tự quay 321.
- Hệ thống ENU (XYZ tương ứng East-North-Up, được sử dụng trong nước, tương ứng với thứ tự quay 312.

<p align="left"><img width="500", src="https://pic2.zhimg.com/80/v2-0808801fe64e9b79d62a6b369a8a6321_720w.jpg"></p>

Hệ tọa độ ENU: Ở đây có thể thấy rằng hệ n không cố định mà quay theo chuyển động quay của trái đất

<p align="left"><img width="350", src="https://pic2.zhimg.com/80/v2-bc6ee3e13875750153b4c4714768fc39_720w.jpg"></p>

Hệ tọa độ NED: Ở đây có thể thấy rằng hệ n không cố định mà quay theo chuyển động quay của trái đất

<p align="left"><img width="350", src="https://pic1.zhimg.com/80/v2-fa88c778be45410c1efea61910f05aa4_720w.jpg"></p>



Công thức chuyển đổi lẫn nhau của góc Euler và ma trận quay theo thứ tự 312 và 321

In [1]:
import numpy as np

sin = np.sin
cos = np.cos
asin = np.arcsin
atan2 = np.arctan2

In [2]:
def ch_eul2m_312(att):
    alpha = att[2]     # (Alpha Z)
    beta = att[0]      # (Beta X)
    gamma = att[1]     # (Gamma Y)

    s_z_alpha = sin(alpha); s_x_beta = sin(beta); s_y_gamma = sin(gamma)
    c_z_alpha = cos(alpha); c_x_beta = cos(beta); c_y_gamma = cos(gamma)

    Cb2n_312 = np.array([[ c_y_gamma*c_z_alpha-s_x_beta*s_y_gamma*s_z_alpha, -c_x_beta*s_z_alpha,  s_y_gamma*c_z_alpha+s_x_beta*c_y_gamma*s_z_alpha],
                         [c_y_gamma*s_z_alpha+s_x_beta*s_y_gamma*c_z_alpha,  c_x_beta*c_z_alpha,  s_y_gamma*s_z_alpha-s_x_beta*c_y_gamma*c_z_alpha],
                         [-c_x_beta*s_y_gamma,           s_x_beta,     c_x_beta*c_y_gamma           ]])
    return Cb2n_312

def ch_eul2m_321(att):
    alpha = att[2]     # (Alpha Z)
    beta = att[0]      # (Beta X)
    gamma = att[1]     # (Gamma Y)

    s_z_alpha = sin(alpha); s_x_beta = sin(beta); s_y_gamma = sin(gamma)
    c_z_alpha = cos(alpha); c_x_beta = cos(beta); c_y_gamma = cos(gamma)

    Cb2n_321 = np.array([[ c_y_gamma*c_z_alpha, s_x_beta*s_y_gamma*c_z_alpha-c_x_beta*s_z_alpha, c_x_beta*s_y_gamma*c_z_alpha+s_x_beta*s_z_alpha],
                         [c_y_gamma*s_z_alpha, s_x_beta*s_y_gamma*s_z_alpha+c_x_beta*c_z_alpha, c_x_beta*s_y_gamma*s_z_alpha-s_x_beta*c_z_alpha],
                         [-s_y_gamma,    s_x_beta*c_y_gamma,          c_x_beta*c_y_gamma            ]])
    return Cb2n_321

In [4]:
def ch_m2eul_312(Cb2n):
    eul_312 = np.array([asin(Cb2n[2,1]),
                        atan2(-Cb2n[2,0], Cb2n[2,2]),
                        atan2(-Cb2n[0,1], Cb2n[1,1])])
    return eul_312

def ch_m2eul_321(Cb2n):
    eul_321 = np.array([atan2(Cb2n[2,1], Cb2n[2,2]),
                        asin(-Cb2n[2,0]),
                        atan2(Cb2n[1,0],Cb2n[0,0])])
    return eul_321

In [8]:
eul_312 = [30, 40, 50]  # [pitch(Around the X axis) roll(Around the Y axis)  yaw(Around the Z axis)]  #312
print("Rotate in the order of 312 (first turn Z-then X-last Y) \nwhere the rotation angle of X, Y, Z is %.3f° %.3f° %.3f° \nto get the corresponding coordinate transformation matrix:" %(eul_312[0], eul_312[1], eul_312[2]))
eul_312rad = np.deg2rad(eul_312)
Cb2n_312 = ch_eul2m_312(eul_312rad)
print(Cb2n_312)

print("Convert Cb2n_312 back to Euler angles:")
eul_312 = ch_m2eul_312(Cb2n_312)
eul_312 = np.rad2deg(eul_312)
print("Get Euler angles in 312 order: %.3f°(Pitch) %.3f°(Roll) %.3f°(Yaw)\n" %(eul_312[0], eul_312[1], eul_312[2]))

Rotate in the order of 312 (first turn Z-then X-last Y) 
where the rotation angle of X, Y, Z is 30.000° 40.000° 50.000° 
to get the corresponding coordinate transformation matrix:
[[ 0.24620194 -0.66341395  0.70658796]
 [ 0.79341204  0.5566704   0.24620194]
 [-0.5566704   0.5         0.66341395]]
Convert Cb2n_312 back to Euler angles:
Get Euler angles in 312 order: 30.000°(Pitch) 40.000°(Roll) 50.000°(Yaw)



Kết quả chuyển đổi có thể được so sánh và xác minh với kết quả trong bài viết trước. Cho đến nay, chúng ta đã thực hiện phép toán quay giữa các góc Euler và ma trận quay.

Vấn đề bế tắc Euler nổi tiếng:

Còn được gọi là vấn đề khóa gimbal góc Euler. Bất kỳ cách biểu diễn nào chỉ sử dụng ba biến số để thể hiện tư thế cơ thể cứng nhắc sẽ gặp một vấn đề bế tắc, đây là một vấn đề toán học và không thể tránh khỏi. Nó có thể được giải quyết bằng quaternion hoặc biểu diễn bằng nhiều biến số hơn (chẳng hạn như DCM).

<p align="left"><img width="350", src="https://pic4.zhimg.com/v2-43336ee9ad6856f7ed593b5394c44d43_b.webp"></p>